In [1]:
import os
from clickhouse_driver import Client
import logging
from datetime import timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'
agregat_table_name = 'Agregat'
olap_table_name = 'OlapCube_VNV'

# Функция для создания основного OLAP куба без заполнения mi8t_count и mi17_count
def create_multidimensional_olap_cube(client):
    # Получение уникальных значений serialno из таблицы Vygruzka
    try:
        serialnos_query = f"""
            SELECT DISTINCT serialno 
            FROM {database_name}.{vygruzka_table_name}
        """
        serialnos = client.execute(serialnos_query)
        serialnos = [row[0] for row in serialnos]
        logging.info(f"Количество загруженных агрегатов: {len(serialnos)}")
    except Exception as e:
        logging.error(f"Ошибка при получении serialno: {e}")
        raise

    # Получение начальной даты из таблицы Agregat
    try:
        unload_date_query = f"""
            SELECT MIN(UnloadDate) 
            FROM {database_name}.{agregat_table_name} 
            WHERE UnloadDate IS NOT NULL
        """
        unload_date = client.execute(unload_date_query)[0][0]
        logging.info(f"Начальная дата: {unload_date}")
        if unload_date is None:
            raise ValueError("Не удалось получить начальную дату из таблицы Agregat")
    except Exception as e:
        logging.error(f"Ошибка при получении начальной даты: {e}")
        raise

    # Создание диапазона дат +4000 дней начиная с unload_date
    date_range = [unload_date + timedelta(days=i) for i in range(4001)]
    logging.info(f"Диапазон дат создан: {len(date_range)} дней")

    # Создание таблицы для OLAP куба
    try:
        # Удаление таблицы только если она существует
        client.execute(f"DROP TABLE IF EXISTS {database_name}.{olap_table_name}")
        logging.info(f"Существующая таблица {olap_table_name} удалена (если существовала)")
    except Exception as e:
        logging.error(f"Ошибка при удалении таблицы {olap_table_name}: {e}")
        raise

    create_olap_query = f"""
    CREATE TABLE IF NOT EXISTS {database_name}.{olap_table_name} (
        serialno String,
        Dates Date,
        Status_P Nullable(String),
        repair_days Nullable(Float64),
        sne Nullable(Float64),
        ppr Nullable(Float64),
        Status Nullable(String),
        daily_flight_hours Nullable(Float64),
        daily_flight_hours_f Nullable(Float64),
        BR Nullable(Float64),
        ll Nullable(Float64),
        oh Nullable(Float64),
        threshold Nullable(Float64),
        Effectivity Nullable(String),
        mi8t_count Nullable(Float64),
        mi17_count Nullable(Float64),
        balance_mi8t Float64,
        balance_mi17 Float64,
        balance_total Float64,
        stock_mi8t Float64,
        stock_mi17 Float64,
        stock_total Float64,
        balance_empty Float64,
        stock_empty Float64,
        location String,         
        ac_typ String            
    ) ENGINE = ReplacingMergeTree()
    ORDER BY (serialno, Dates)
    """
    try:
        client.execute(create_olap_query)
        logging.info(f"Таблица {olap_table_name} успешно создана с ReplacingMergeTree")
    except Exception as e:
        logging.error(f"Ошибка при создании таблицы {olap_table_name}: {e}")
        raise

    # Подготовка данных для вставки в OLAP куб без заполнения mi8t_count и mi17_count
    olap_data = []
    total_records = len(serialnos) * len(date_range)
    logging.info(f"Начинается подготовка данных для вставки: {total_records} записей")

    for idx, serialno in enumerate(serialnos, start=1):
        if idx % 10 == 0:
            logging.info(f"Подготовлено {idx}/{len(serialnos)} serialno")

        for date in date_range:
            # Здесь мы не заполняем mi8t_count и mi17_count, устанавливаем их в NULL
            olap_data.append((
                serialno,
                date,
                None,  # Status_P
                None,  # repair_days
                None,  # sne
                None,  # ppr
                None,  # Status
                None,  # daily_flight_hours
                None,  # daily_flight_hours_f
                None,  # BR
                None,  # ll
                None,  # oh
                None,  # threshold
                None,  # Effectivity
                None,  # mi8t_count
                None,  # mi17_count
                0.0,  # balance_mi8t
                0.0,  # balance_mi17
                0.0,  # balance_total
                0.0,  # stock_mi8t
                0.0,  # stock_mi17
                0.0,  # stock_total
                0.0,  # balance_empty
                0.0,  # stock_empty
                '',   # location
                ''    # ac_typ
            ))

            # Пакетная вставка каждые 100000 записей
            if len(olap_data) >= 100000:
                try:
                    client.execute(
                        f"INSERT INTO {database_name}.{olap_table_name} VALUES",
                        olap_data
                    )
                    logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
                    olap_data = []
                except Exception as e:
                    logging.error(f"Ошибка при вставке данных в {olap_table_name}: {e}")
                    raise

    # Вставка оставшихся записей
    if olap_data:
        try:
            client.execute(
                f"INSERT INTO {database_name}.{olap_table_name} VALUES",
                olap_data
            )
            logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
        except Exception as e:
            logging.error(f"Ошибка при вставке оставшихся данных в {olap_table_name}: {e}")
            raise

    logging.info("Все данные успешно загружены в основной OLAP куб")

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name  # Указание базы данных
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Создание и загрузка OLAP куба
try:
    create_multidimensional_olap_cube(client)
    logging.info("Многомерный OLAP куб успешно создан и данные загружены")
except Exception as e:
    logging.error(f"Произошла ошибка при создании OLAP куба: {e}", exc_info=True)


2025-03-04 14:52:35,613 - INFO - Подключение к ClickHouse успешно установлено
2025-03-04 14:52:35,630 - INFO - Количество загруженных агрегатов: 185
2025-03-04 14:52:35,640 - INFO - Начальная дата: 2024-11-25
2025-03-04 14:52:35,645 - INFO - Диапазон дат создан: 4001 дней
2025-03-04 14:52:35,652 - INFO - Существующая таблица OlapCube_VNV удалена (если существовала)
2025-03-04 14:52:35,668 - INFO - Таблица OlapCube_VNV успешно создана с ReplacingMergeTree
2025-03-04 14:52:35,669 - INFO - Начинается подготовка данных для вставки: 740185 записей
2025-03-04 14:52:35,694 - INFO - Подготовлено 10/185 serialno
2025-03-04 14:52:35,753 - INFO - Подготовлено 20/185 serialno
2025-03-04 14:52:37,652 - INFO - Вставлено 100000 записей в OlapCube_VNV
2025-03-04 14:52:37,674 - INFO - Подготовлено 30/185 serialno
2025-03-04 14:52:37,698 - INFO - Подготовлено 40/185 serialno
2025-03-04 14:52:37,723 - INFO - Подготовлено 50/185 serialno
2025-03-04 14:52:39,562 - INFO - Вставлено 100000 записей в OlapCube

In [2]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'
olap_table_name = 'OlapCube_VNV'

# Функция для получения данных из таблицы Vygruzka для OLAP куба
def get_vygruzka_data(client):
    try:
        vygruzka_query = f"""
            SELECT serialno, ac_typ, BR, ll, oh, threshold, Effectivity
            FROM {database_name}.{vygruzka_table_name}
            WHERE serialno IS NOT NULL
        """
        vygruzka_results = client.execute(vygruzka_query)
        vygruzka_dict = {row[0]: row[1:] for row in vygruzka_results}
        logging.info(f"Получено данных из Vygruzka: {len(vygruzka_dict)} записей")
        return vygruzka_dict
    except Exception as e:
        logging.error(f"Ошибка при получении данных из Vygruzka: {e}")
        raise

# Функция для дополнения таблицы OLAP новыми полями, если они отсутствуют
def alter_olap_table(client, olap_table_name):
    try:
        alter_query = f"""
        ALTER TABLE {database_name}.{olap_table_name}
        ADD COLUMN IF NOT EXISTS ac_typ String,
        ADD COLUMN IF NOT EXISTS BR Float64,
        ADD COLUMN IF NOT EXISTS ll Float64,
        ADD COLUMN IF NOT EXISTS oh Float64,
        ADD COLUMN IF NOT EXISTS threshold Float64,
        ADD COLUMN IF NOT EXISTS Effectivity String
        """
        client.execute(alter_query)
        logging.info(f"Таблица {olap_table_name} успешно дополнена новыми полями")
    except Exception as e:
        logging.error(f"Ошибка при изменении таблицы {olap_table_name}: {e}")
        raise

# Функция для обновления OLAP куба новыми полями
def update_olap_cube(client, vygruzka_dict):
    try:
        # Подготовка данных для обновления
        # В ClickHouse лучше обновлять данные пакетами, но оператор ALTER TABLE ... UPDATE выполняется построчно
        # Поэтому для больших объемов данных этот метод может быть медленным
        for idx, (serialno, data) in enumerate(vygruzka_dict.items(), start=1):
            if idx % 100 == 0:
                logging.info(f"Обработано {idx}/{len(vygruzka_dict)} serialno")

            ac_typ, BR, ll, oh, threshold, Effectivity = data

            # Приведение типов и обработка значений
            BR = float(BR) if BR is not None else 0.0
            ll = float(ll) if ll is not None else 0.0
            oh = float(oh) if oh is not None else 0.0
            threshold = float(threshold) if threshold is not None else 0.0
            ac_typ = ac_typ if ac_typ is not None else ""
            Effectivity = Effectivity if Effectivity is not None else ""

            # Выполнение UPDATE для всех записей с данным serialno
            update_query = f"""
                ALTER TABLE {database_name}.{olap_table_name}
                UPDATE 
                    ac_typ = %(ac_typ)s,
                    BR = %(BR)s,
                    ll = %(ll)s,
                    oh = %(oh)s,
                    threshold = %(threshold)s,
                    Effectivity = %(Effectivity)s
                WHERE serialno = %(serialno)s
            """
            params = {
                'ac_typ': ac_typ,
                'BR': BR,
                'll': ll,
                'oh': oh,
                'threshold': threshold,
                'Effectivity': Effectivity,
                'serialno': serialno
            }
            client.execute(update_query, params)
    except Exception as e:
        logging.error(f"Ошибка при обновлении OLAP куба: {e}")
        raise
    logging.info("Все данные успешно обновлены в OLAP кубе")

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name  # Добавлено указание базы данных
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Дополнение OLAP куба новыми полями
try:
    alter_olap_table(client, olap_table_name)
    vygruzka_dict = get_vygruzka_data(client)
    update_olap_cube(client, vygruzka_dict)
    logging.info("Многомерный OLAP куб успешно дополнен новыми данными")
except Exception as e:
    logging.error(f"Произошла ошибка при дополнении OLAP куба: {e}", exc_info=True)

2025-03-04 14:52:49,932 - INFO - Подключение к ClickHouse успешно установлено
2025-03-04 14:52:49,949 - INFO - Таблица OlapCube_VNV успешно дополнена новыми полями
2025-03-04 14:52:49,959 - INFO - Получено данных из Vygruzka: 185 записей


2025-03-04 14:52:51,555 - INFO - Обработано 100/185 serialno
2025-03-04 14:52:53,046 - INFO - Все данные успешно обновлены в OLAP кубе
2025-03-04 14:52:53,047 - INFO - Многомерный OLAP куб успешно дополнен новыми данными


In [3]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'

# Функция для проверки существования колонки в таблице
def check_column_exists(client, table_name, column_name):
    try:
        query = f"""
            SELECT name FROM system.columns
            WHERE database = '{database_name}' AND table = '{table_name}' AND name = '{column_name}'
        """
        result = client.execute(query)
        return len(result) > 0
    except Exception as e:
        logging.error(f"Ошибка при проверке существования колонки {column_name} в таблице {table_name}: {e}")
        raise

# Функция для обновления данных в таблице OLAP на основе данных из Vygruzka с фильтрацией по location, содержащему "RA-"
def update_location_data(client):
    olap_table_name = 'OlapCube_VNV'
    column_name = 'location'

    # Проверка существования колонки location в таблице OlapCube_VNV
    if not check_column_exists(client, olap_table_name, column_name):
        logging.error(f"Колонка {column_name} не существует в таблице {olap_table_name}")
        return

    # Получение данных из таблицы Vygruzka с фильтрацией по location, содержащему "RA-"
    try:
        vygruzka_query = f"""
            SELECT serialno, location
            FROM {database_name}.{vygruzka_table_name}
            WHERE serialno IS NOT NULL AND location LIKE 'RA-%'
        """
        vygruzka_results = client.execute(vygruzka_query)
        logging.info(f"Получено данных из Vygruzka: {len(vygruzka_results)} записей, содержащих 'RA-' в поле location")
    except Exception as e:
        logging.error(f"Ошибка при получении данных из Vygruzka: {e}")
        raise

    # Обновление таблицы OLAP на основе данных из Vygruzka
    try:
        for serialno, location in vygruzka_results:
            update_query = f"""
                ALTER TABLE {database_name}.{olap_table_name} UPDATE location = '{location}'
                WHERE serialno = '{serialno}'
            """
            client.execute(update_query)
        logging.info(f"Данные о местоположении успешно обновлены в {olap_table_name}")
    except Exception as e:
        logging.error(f"Ошибка при обновлении данных в {olap_table_name}: {e}")
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Обновление данных о местоположении в OLAP кубе
try:
    update_location_data(client)
    logging.info("Данные о местоположении успешно обновлены в OLAP кубе")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении данных о местоположении: {e}", exc_info=True)


2025-03-04 14:52:53,072 - INFO - Подключение к ClickHouse успешно установлено
2025-03-04 14:52:53,093 - INFO - Получено данных из Vygruzka: 162 записей, содержащих 'RA-' в поле location
2025-03-04 14:52:55,381 - INFO - Данные о местоположении успешно обновлены в OlapCube_VNV
2025-03-04 14:52:55,383 - INFO - Данные о местоположении успешно обновлены в OLAP кубе
